# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Уменьшение размеров модели
### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 баллов__.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн 29.11.24 23:59__ \
__Жесткий дедлайн 2.12.24 23:59__

### О задании

В этом задании вам предстоит научиться решать задачу Named Entity Recognition (NER) на самом популярном датасете – [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). В вашем распоряжении будет предобученный BERT, который вам необходимо уменьшить с минимальными потерями в качестве до размера 20М параметров. Для этого вы самостоятельно реализуете факторизацию эмбеддингов, дистилляцию, шеринг параметров и так далее.

В этом задании вам придется проводить довольно много экспериментов, поэтому мы рекомендуем не писать весь код в тетрадке, а завести разные файлы для отдельных логических блоков и скомпоновать все в виде проекта. Это позволит вашему ноутбуку не разрастаться и сильно облегчит задачу и вам, и проверяющим. Так же постарайтесь логгировать все ваши эксперименты в wandb, чтобы ничего не потерялось.

### Оценивание
Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором, как и раньше, от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета можно потерять баллы за сами задания. Задания делятся на две части: _номерные_ и _на выбор_. За _номерные_ можно получить в сумме 6 баллов, за задания _на выбор_ можно получить до 16. То есть за все дз можно получить 24 балла. Все, что вы наберете свыше 10, будет считаться бонусами.


### О датасете

Named Entity Recognition – это задача классификации токенов по классам сущностей. В CoNLL-2003 для именования сущностей используется маркировка **BIO** (Beggining, Inside, Outside), в которой метки означают следующее:

- *B-{метка}* – начало сущности *{метка}*
- *I-{метка}* – продолжнение сущности *{метка}*
- *O* – не сущность

Существуют так же и другие способы маркировки, например, BILUO. Почитать о них можно [тут](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) и [тут](https://www.youtube.com/watch?v=dQw4w9WgXcQ).

Всего в датасете есть 9 разных меток.
- O – слову не соответствует ни одна сущность.
- B-PER/I-PER – слово или набор слов соответстует определенному _человеку_.
- B-ORG/I-ORG – слово или набор слов соответстует определенной _организации_.
- B-LOC/I-LOC – слово или набор слов соответстует определенной _локации_.
- B-MISC/I-MISC – слово или набор слов соответстует сущности, которая не относится ни к одной из предыдущих. Например, национальность, произведение искусства, мероприятие и т.д.

Приступим!

In [57]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# Определим расположение проекта

import sys
import os

project_path = '/content/drive/MyDrive/Bert-NER-Quantization-5/'
sys.path.append(project_path)

os.chdir(project_path)

In [59]:
# Авторизация в сервисах

from huggingface_hub import login
import wandb
import yaml

with open('auth_keys.yaml', 'r') as f:
    config = yaml.safe_load(f)

login(token=config["hf_token"])
wandb.login(key=config['wandb_token'])

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [60]:
# Установка зависимостей

!pip -q install -r requirements.txt

Начнем с загрузки и предобработки датасета.

In [61]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [62]:
from datasets import load_dataset

dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)

dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3453
    })
})

In [63]:
# An example of 'train' looks as follows.

# {
#     "chunk_tags": [11, 12, 12, 21, 13, 11, 11, 21, 13, 11, 12, 13, 11, 21, 22, 11, 12, 17, 11, 21, 17, 11, 12, 12, 21, 22, 22, 13, 11, 0],
#     "id": "0",
#     "ner_tags": [0, 3, 4, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#     "pos_tags": [12, 22, 22, 38, 15, 22, 28, 38, 15, 16, 21, 35, 24, 35, 37, 16, 21, 15, 24, 41, 15, 16, 21, 21, 20, 37, 40, 35, 21, 7],
#     "tokens": ["The", "European", "Commission", "said", "on", "Thursday", "it", "disagreed", "with", "German", "advice", "to", "consumers", "to", "shun", "British", "lamb", "until", "scientists", "determine", "whether", "mad", "cow", "disease", "can", "be", "transmitted", "to", "sheep", "."]
# }


In [64]:
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [65]:
words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]

for i in range(len(words)):
    print(f'{words[i]}\t{label_names[labels[i]]}')

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


### Предобработка

На протяжении всего домашнего задания мы будем использовать _cased_ версию BERT, то есть токенизатор будет учитывать регистр слов. Для задачи NER регистр важен, так как имена и названия организаций или предметов искусства часто пишутся с большой буквы, и будет глупо прятать от модели такую информацию.

In [66]:
# Загрузка функций из файла custom_tokenize.py

from custom_tokenize import tokenizer, align_labels_and_tokens, tokenize_dataset

При токенизации слова могут разделиться на несколько токенов (как слово `Fischler` из примера ниже), из-за чего появится несоответствие между числом токенов и меток. Это несоответствие нам придется устранить вручную.

In [67]:
example = dataset["train"][12]
words = example["tokens"]
tags = [label_names[t] for t in example["ner_tags"]]
tokenized_text = tokenizer(example["tokens"], is_split_into_words=True)

print('Слова: ', words)
print('Токены:', tokenized_text.tokens())
print('Метки:', tags)


Слова:  ['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
Токены: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Метки: ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']


__Задание 1 (1 балл).__ Токенизируйте весь датасет и для каждого текста выравните токены с метками так, чтобы каждому токену соответствовала одна метка. При этом важно сохранить нотацию BIO. И не забудьте про специальные токены! Должно получиться что-то такое:

In [68]:
# aligned_labels = align_labels_with_tokens(example["ner_tags"], tokenized_text)
aligned_labels = align_labels_and_tokens(tokenized_text.word_ids(), example["ner_tags"])

tags = [label_names[t] if t > -1 else t for t in aligned_labels]

print("Токены:", tokenized_text.tokens())
print("Выровненные метки:", aligned_labels)
print("Выровненные названия меток:", tags)

Токены: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Выровненные метки: [-100, 0, 5, 0, 5, 0, 1, 2, 2, 0, 0, 0, 0, -100]
Выровненные названия меток: [-100, 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', -100]


In [69]:
# Исходное предложение №12

print(dataset['train'][12:13]['tokens'][0])

['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']


In [70]:
# Токены после применения Bert tokenizer

print(tokenize_dataset(dataset['train'][12:13]).tokens())

['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']


In [71]:
# Выровненные метки

print(tokenize_dataset(dataset['train'][12:13])['labels'])

[[-100, 0, 5, 0, 5, 0, 1, 2, 2, 0, 0, 0, 0, -100]]


In [72]:
# Выровненные названия меток

tags = [label_names[t] if t > -1 else t for t in tokenize_dataset(dataset['train'][12:13])['labels'][0]]
print(tags)

[-100, 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', -100]


In [73]:
# Вывод: выравнивание меток работает, можно токенизировать весь датасет

tokenized_dataset = dataset.map(
    tokenize_dataset,
    batched = True
)

tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

### Метрика

Для оценки качества NER обычно используют F1 меру с микро-усреднением. Мы загрузим ее из библиотеки `seqeval`. Функция `f1_score` принимает два 2d списка с правильными и предсказанными метками, записаными текстом, и возвращает для них значение F1. Вы можете использовать ее с параметрами по умолчанию.

In [74]:
from seqeval.metrics import f1_score

Особенность подсчета F1 для NER заключается в том, что в некоторых ситуациях неправильные ответы могут засчитываться как правильные. Например, если модель предсказала `['I-PER', 'I-PER']`, то мы можем догадаться, что на самом деле должно быть `['B-PER', 'I-PER']`, так как сущность не может начинаться с `I-`. Функция `f1_score` учитывает это и поэтому работает только с текстовыми представлениями меток.

### Модель

В качестве базовой модели мы возьмем `bert-base-cased`. Как вы понимаете, он не обучался на задачу NER. Поэтому прежде чем приступать к уменьшению размера BERT, его необходимо дообучить.

__Задание 2 (1 балл)__ Дообучите `bert-base-cased` на нашем датасете с помощью обычного fine-tuning. У вас должно получиться хотя бы 0.9 F1 на тестовой выборке. Заметьте, что чем выше качество большой модели, тем лучше будет работать дистиллированный ученик. Для обучения можно использовать `Trainer` из Hugging Face.

In [75]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification

model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_names))

print('Число параметров:', sum(p.numel() for p in model.parameters()))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Число параметров: 107726601


In [76]:
import numpy as np
from custom_metrics import compute_f1_metrics

wandb.init(
    project="Bert_NER_Quantization",
    name="6-epoch",
)

data_collator = DataCollatorForTokenClassification(
    tokenizer = tokenizer
)


BATCH_SIZE = 64
NUM_TRAIN_EPOCH = 10
LR = 2e-5
WEIGHT_DECAY = 0.01

args = TrainingArguments(
    output_dir=project_path,
    evaluation_strategy = "epoch",
    report_to="wandb",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_TRAIN_EPOCH,
    weight_decay=WEIGHT_DECAY,
    save_strategy = "epoch"
)

trainer = Trainer(
    model,
    args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    # train_dataset=tokenized_dataset["train"].select(range(10)),
    # eval_dataset=tokenized_dataset["validation"].select(range(10)),
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_f1_metrics
)

# trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-76-6cb8865f817f>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Training and Validation Results

| Epoch | Training Loss | Validation Loss | F1       |
|-------|---------------|-----------------|----------|
| 1     | No log        | 0.074044        | 0.892728 |
| 2     | No log        | 0.063494        | 0.921673 |
| 3     | 0.150600      | 0.056858        | 0.935454 |
| 4     | 0.150600      | 0.055022        | 0.933809 |
| 5     | 0.027000      | 0.061152        | 0.934972 |
| 6     | 0.027000      | 0.063808        | 0.934916 |
| 7     | 0.012900      | 0.065230        | 0.937536 |
| 8     | 0.012900      | 0.062988        | 0.940451 |
| 9     | 0.012900      | 0.067069        | 0.940481 |
| 10    | 0.007400      | 0.066939        | 0.941343 |


In [77]:
from transformers import BertConfig

# Загрузка лучшей модели №4, потому что дальше начинается переобучение

best_model_path = "checkpoint-880"
model_tuned = AutoModelForTokenClassification.from_pretrained(best_model_path)

model_tuned.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

### Факторизация матрицы эмбеддингов

Можно заметить, что на данный момент матрица эмбеддингов занимает $V \cdot H = 28996 \cdot 768 = 22.268.928$ параметров. Это aж пятая часть от всей модели! Давайте попробуем что-то с этим сделать. В модели [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) предлагается факторизовать матрицу эмбеддингов в произведение двух небольших матриц. Таким образом, параметры эмбеддингов будут содержать $V \cdot E + E \cdot H$ элементов, что гораздо меньше $V \cdot H$, если $H \gg E$. Авторы выбирают $E = 128$, однако ничего не мешает нам взять любое другое значение. Например, выбрав $E = 64$, мы уменьшим число параметров примерно на 20М.

__Задание 3 (1 балл).__ Напишите класс-обертку над слоем эмбеддингов, который реализует факторизацию на две матрицы, и дообучите факторизованную модель. Заметьте, обе матрицы можно инициализировать с помощью SVD разложения, чтобы начальное приближение было хорошим. Это сэкономит очень много времени на дообучении. С рангом разложения $E = 64$ у вас должно получиться F1 больше 0.87.

In [78]:
from factorization import FactorizedEmbedding

# Получаем обученную матрицу эмбеддингов
pretrained_embedding = model_tuned.bert.embeddings.word_embeddings.weight.data

embedding_dim = 64  # E

factorized_embedding = FactorizedEmbedding(pretrained_embedding, embedding_dim)

# Вставляем новую матрицу эмбеддингов
model_tuned.bert.embeddings.word_embeddings = factorized_embedding
model_tuned_fuct = model_tuned

print('Число параметров:', sum(p.numel() for p in model_tuned.parameters()))

Число параметров: 87363337


In [79]:
print('Число параметров в уменьшенной модели:', sum(p.numel() for p in model_tuned.parameters()))
print('Число параметров в исходной модели:', sum(p.numel() for p in model.parameters()))
print('Модель стала легче на:', (sum(p.numel() for p in model.parameters()) - sum(p.numel() for p in model_tuned.parameters())), )

Число параметров в уменьшенной модели: 87363337
Число параметров в исходной модели: 107726601
Модель стала легче на: 20363264


In [80]:
# wandb.init(
#     project="Bert_NER_Quantization_fact_emb",
#     name="10-epoch"
# )

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

BATCH_SIZE = 64
NUM_TRAIN_EPOCH = 10
LR = 2e-5
WEIGHT_DECAY = 0.01


args = TrainingArguments(
    output_dir="factorized_bert",
    evaluation_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_TRAIN_EPOCH,
    weight_decay=WEIGHT_DECAY,
    save_strategy="epoch",
    report_to="wandb"
)


trainer = Trainer(
    model=model_tuned_fuct,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_f1_metrics
)

# trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-80-2b3b0de62067>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Training and Validation Results Factorized

| Epoch | Training Loss | Validation Loss | F1       |
|-------|---------------|-----------------|----------|
| 1     | No log        | 0.193183        | 0.770092 |
| 2     | No log        | 0.131908        | 0.846841 |
| 3     | 0.276100      | 0.112845        | 0.862722 |
| 4     | 0.276100      | 0.103340        | 0.874274 |
| 5     | 0.112000      | 0.095571        | 0.885186 |
| 6     | 0.112000      | 0.091554        | 0.887709 |
| 7     | 0.079600      | 0.092220        | 0.891708 |
| 8     | 0.079600      | 0.089843        | 0.895916 |
| 9     | 0.079600      | 0.088674        | 0.898594 |
| 10    | 0.060300      | 0.089163        | 0.900424 |


In [81]:
# Выходим на плато ошибки примерно на 5-й эпохе с f1 = 0.88

### Дистилляция знаний

Дистилляция знаний – это парадигма обучения, в которой знания модели-учителя дистиллируются в модель-ученика. Учеником может быть произвольная модель меньшего размера, решающая ту же задачу, однако обычно ученик имеет ту же архитектуру, что и учитель. При дистилляции используются два функционала ошибки:

1. Стандартная кросс-энтропия.
1. Функция, задающая расстояние между распределениями предсказаний учителя и ученика. Чаще всего используют KL-дивергенцию.

Для того, чтобы распределение предсказаний учителя не было вырожденным, к softmax добавляют температуру больше 1, например, 2 или 5.   
__Важно:__ при делении логитов на температуру значения градиентов уменьшаются в $\tau^2$ раз (проверьте это!). Поэтому для возвращения их в изначальный масштаб ошибку надо домножить на $\tau^2$. Подробнее об этом можно почитать в разделе 2.1 [оригинальной статьи](https://arxiv.org/pdf/1503.02531).

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png" width="1000">

__Задание 4 (3 балла).__ Реализуйте метод дистилляции знаний, изображенный на картинке. Для подсчета ошибки между предсказаниями ученика и учителя используйте KL-дивергенцию [`nn.KLDivLoss(reduction="batchmean")`](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html) (обратите внимание на вормат ее входов). Для получения итоговой ошибки суммируйте мягкую ошибку с жесткой.   
В качестве учителя используйте дообученный BERT из задания 2. В качестве ученика возьмите необученную модель с размером __не больше 20M__ параметров. Вы можете использовать факторизацию матрицы эмбеддингов для уменьшения числа параметров. Если вы все сделали правильно, то на тестовой выборке вы должны получить значение F1 не меньше 0.7. Вам должно хватить примерно 20к итераций обучения для этого. Если у вас что-то не получается, то можно ориентироваться на статью про [DistilBERT](https://arxiv.org/abs/1910.01108) и на [эту статью](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT).

__Важно:__
* Не забывайте добавлять _warmup_ при обучении ученика.
* Не забывайте переводить учителя в режим _eval_.

In [82]:
# Загрузка небольшого ученика TinyBERT

from transformers import AutoConfig, AutoTokenizer

def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print()
    print("Количество парметров у ученика")
    print(f"Total parameters: {total_params}")
    print(f"Trainable parameters: {trainable_params}")
    print(f"Non-trainable parameters: {total_params - trainable_params}")


# Загрузка токенизатора
tokenizer_student = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

# Загрузка модели
model_student = AutoModelForTokenClassification.from_pretrained(
    "huawei-noah/TinyBERT_General_4L_312D",
    num_labels=len(label_names)
)

# Просмотр количества параметров
count_parameters(model_student)
print()

# Просмотр структуры модели
model_student

Some weights of BertForTokenClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Количество парметров у ученика
Total parameters: 14255409
Trainable parameters: 14255409
Non-trainable parameters: 0



BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, 

In [83]:
# Обучение ученика без учителя, чтобы зафиксировать бейзлайн

wandb.init(
    project="Bert_NER_Student",
)

data_collator_student = DataCollatorForTokenClassification(tokenizer=tokenizer_student)

BATCH_SIZE = 64
NUM_TRAIN_EPOCH = 20
LR = 2e-5
WEIGHT_DECAY = 0.01

args_student = TrainingArguments(
    output_dir="student_bert",
    evaluation_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_TRAIN_EPOCH,
    weight_decay=WEIGHT_DECAY,
    save_strategy="epoch",
    report_to="wandb"
)


trainer_student = Trainer(
    model=model_student,
    args=args_student,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator_student,
    tokenizer=tokenizer_student,
    compute_metrics=compute_f1_metrics
)

# # trainer_student.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-83-25e1a173d8d8>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_student = Trainer(


**Комментарий:**

Достигаем нужной метрики примерно на 15-й эпохе

In [84]:
# Измененный Trainer
from distillation import DistillationTrainer

# Загрузка предобученного учителя
teacher_model = AutoModelForTokenClassification.from_pretrained(best_model_path)

# Перевод учителя в режим инференса
teacher_model = teacher_model.eval().to(device)

# Загрузка ученика
student_model = AutoModelForTokenClassification.from_pretrained(
    "huawei-noah/TinyBERT_General_4L_312D",
    num_labels=len(label_names)
)

# Загрузка токенизатора ученика
tokenizer_student = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")

data_collator_student = DataCollatorForTokenClassification(tokenizer=tokenizer_student)

# Аргементы запуска обучения
BATCH_SIZE = 64
NUM_TRAIN_EPOCH = 10
LR = 2e-5
WEIGHT_DECAY = 0.01
TEMPERATURE = 2.0
ALPHA = 0.5

EVAL_STRATEG = "steps"
EVAL_STEPS = 500
MAX_STEPS = 2000


args = TrainingArguments(
    output_dir="distilled_tinybert",
    evaluation_strategy=EVAL_STRATEG,
    eval_steps=EVAL_STEPS,
    max_steps=MAX_STEPS,
    warmup_steps=500,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=WEIGHT_DECAY,
)


trainer = DistillationTrainer(
    teacher_model=teacher_model,
    temperature=2.0,
    alpha=0.5,
    model=student_model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator_student,
    tokenizer=tokenizer_student,
    compute_metrics=compute_f1_metrics,
)

# trainer.train()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/content/drive/MyDrive/Bert-NER-Quantization-5/distillation.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [85]:
# trainer.evaluate()

# Задания на выбор

Как вы понимаете, есть еще довольно много разных способов уменьшить обученную модель. В этой секции вам предлагается реализовать разные техники на выбор. За каждую из них можно получить разное количество балов в зависимости от сложности. Успешность реализации будет оцениваться как по коду, так и по качеству на тестовой выборке. Все баллы за это дз, которые вы наберете сверх 10, будут считаться бонусными.   
В задании 4 вы обучали модель с ограничением числа параметров в 20М. При реализации техник из этой секции придерживайтесь такого же ограничение. Это позволит честно сравнивать методы между собой и делать правильные выводы. Напишите в отчете обо всем, что вы попробовали.

* __Шеринг весов (2 балла).__ В модификации BERT [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) помимо факторизации эмбеддингов предлагается шерить веса между слоями. То есть разные слои используют одни и те же веса. Такая техника эвивалентна применению одного и того же слоя несколько раз. Она позволяет в несколько раз уменьшить число параметров и не сильно потерять в качестве.
* __Факторизация промежуточных слоев (2 балла).__ Если можно факторизовать матрицу эмбеддингов, то и все остальное тоже можно. Для факторизации слоев существует много разных подходов и выбрать какой-то один сложно. Вы можете вдохновляться [этим списком](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), найти в интернете что-то другое или придумать метод самостоятельно. В любом случае в отчете обоснуйте, почему вы решили сделать так как сделали.
* __Приближение промежуточных слоев (2 балла).__ Мы обсуждали, что помимо приближения выходов модели ученика к выходам модели учителя, можно приближать выходы промежуточных слоев. В [этой работе](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) подробно написано, как это можно сделать.
* __Прунинг (4 балла).__ В методе [SparseGPT](https://arxiv.org/abs/2301.00774) предлагается подход, удаляющий веса модели один раз после обучения. При этом оказывается возможным удалить до половины всех весов без потери в качестве. Математика, стоящаяя за техникой, довольно сложная, однако общий подход простой – будем удалять веса в каждом слое по отдельности, при удалении части весов слоя, остальные веса будут перенастраиваться так, чтобы общий выход слоя не изменился.
* __Удаление голов (6 баллов).__ В данный момент мы используем все головы внимания, но ряд исследований показывает, что большинство из них можно выбросить без потери качества. В этой [статье](https://arxiv.org/pdf/1905.09418.pdf) предлагается подход, который добавляет гейты к механизму внимания, которые регулируют, какие головы участвуют в слое, а какие – нет. В процессе обучения гейты настраиваются так, чтобы большинство голов не использовалась. В конце обучения неиспользуемые головы можно удалить. За это задание дается много баллов, потому что в методе довольно сложная математика и подход плохо заводится. Если вы решитесь потратить на него свои силы, то в случае неудачи мы дадим промежуточные баллы, опираясь на отчет.   
__Совет:__ во время обучения внимательно следите за поведением гейтов. Если вы все сделали правильно, то они должны зануляться. Однако зануляются они не всегда сразу, им надо дать время и обучать модель подольше.